<a href="https://colab.research.google.com/github/SihyunPark01/Colab/blob/main/Ch04.%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%B6%84%EC%84%9D%20/%202_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
날짜: 21/11/23
이름: 박시현
내용: 영화 리뷰 감성분석 실습하기
"""

!pip install konlpy

     |████████████████████████████████| 19.4 MB 44.2 MB/s 
     |████████████████████████████████| 86 kB 3.6 MB/s 
     |████████████████████████████████| 448 kB 34.3 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
import pickle, re
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Activation, Dropout, Embedding, LSTM
from keras.preprocessing.sequence import pad_sequences


In [11]:
# 데이터 불러오기
train_data = pd.read_table('/content/drive/MyDrive/Colab Notebooks/file/movie_review_train.txt')
test_data = pd.read_table('/content/drive/MyDrive/Colab Notebooks/file/movie_review_test.txt')
train_data

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [14]:
# 전처리 함수 정의

okt = Okt()
stop_words = ['은', '는', '을', '를', '의', '이', '가', '에게', '하', '것', '들', '에서', '있', '되', '등', '한']

def preprocessing(text):
  # 한글 텍스트 아닌 것 제외시키기
  review_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]','',str(text))

  # 문장 형태소 분석
  review_text = okt.morphs(review_text, stem=True)

  # 불용어(을/를/의...) 제거
  word_review = [w for w in review_text if not w in stop_words]

  return word_review



clean_train_review = []
clean_test_review = []

# 훈련 데이터 전처리
for review in train_data['document']:
  clean_train_review.append(preprocessing(review))

# 검증 데이터 전처리
for review in test_data['document']:
  clean_test_review.append(preprocessing(review))


  clean_train_review[:5]

### 전처리 함수 정의 : 15만개의 데이터를 정제하는 작업

In [ ]:
# 토큰화
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_review)
train_sequences = tokenizer.texts_to_sequences(clean_train_review)
test_sequences = tokenizer.texts_to_sequences(clean_test_review)

word_dic = tokenizer.word_index
word_dic


In [16]:
# 패딩 처리
train_inputs = pad_sequences(train_sequences, maxlen=10, padding = 'post') #댓글이 가장 많은 길이가 10글자
test_inputs = pad_sequences(test_sequences, maxlen=10, padding = 'post')

train_labels = np.array(train_data['label'])
test_labels = np.array(test_data['label'])  #라벨의 결과값은 0 또는 1

print(train_inputs[0])
print(train_labels[0])

[ 53 462  18 263 664   0   0   0   0   0]
0


In [18]:
# 모델 생성
model = Sequential()

model.add(Embedding(30000, 10)) 
model.add(LSTM(128)) #RNN모델에서 장기기억을 위한 순환모델 LSTM 하나 더 추가
model.add(Dense(1, activation='sigmoid')) #0~1사이의 결과값을 출력
model.summary()




Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 10)          300000    
                                                                 
 lstm_1 (LSTM)               (None, 128)               71168     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 371,297
Trainable params: 371,297
Non-trainable params: 0
_________________________________________________________________


In [19]:
# 모델 설정 

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [20]:
# 모델 학습

model.fit(train_inputs,
          train_labels,
          epochs=10,
          batch_size=128,
          validation_split=0.2)

Epoch 1/10
938/938 [==============================] - 19s 12ms/step - loss: 0.4362 - acc: 0.7916 - val_loss: 0.3883 - val_acc: 0.8293
Epoch 2/10
938/938 [==============================] - 10s 10ms/step - loss: 0.3486 - acc: 0.8468 - val_loss: 0.3913 - val_acc: 0.8261
Epoch 3/10
938/938 [==============================] - 9s 10ms/step - loss: 0.3210 - acc: 0.8616 - val_loss: 0.4022 - val_acc: 0.8262
Epoch 4/10
938/938 [==============================] - 10s 10ms/step - loss: 0.2982 - acc: 0.8724 - val_loss: 0.3957 - val_acc: 0.8265
Epoch 5/10
938/938 [==============================] - 9s 10ms/step - loss: 0.2730 - acc: 0.8834 - val_loss: 0.4071 - val_acc: 0.8249
Epoch 6/10
938/938 [==============================] - 10s 10ms/step - loss: 0.2465 - acc: 0.8951 - val_loss: 0.4412 - val_acc: 0.8221
Epoch 7/10
938/938 [==============================] - 10s 10ms/step - loss: 0.2213 - acc: 0.9065 - val_loss: 0.5049 - val_acc: 0.8197
Epoch 8/10
938/938 [==============================] - 10s 10ms/s

In [21]:
# 모델 평가

result = model.evaluate(test_inputs, test_labels)

1563/1563 [==============================] - 6s 4ms/step - loss: 0.6074 - acc: 0.8128


In [23]:
# 테스트
def sentiment_predict(text):
  # 한글 텍스트 아닌 것 제외시키기
  review_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]','',str(text))

  # 문장 형태소 분석
  review_text = okt.morphs(review_text, stem=True)

  # 불용어(을/를/의...) 제거
  word_review = [w for w in review_text if not w in stop_words]

  encoded = tokenizer.texts_to_sequences([word_review])  #문장을 수치화
  padded = pad_sequences(encoded, maxlen=10, padding='post') #패딩값을 채우고


  score = float(model.predict(padded)) #0~1사이의 값으로 리턴
  return score

In [ ]:
while True:
  review = input('리뷰 입력 :')
  score = sentiment_predict(review)

  if score > 0.5:
    print('{:.2f}% 확률로 긍정 리뷰입니다.\n'.format(score * 100))
  else:
    print('{:.2f}% 확률로 부정 리뷰입니다.\n'.format((1-score) * 100))